In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

from scipy import stats
import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('..')
sys.path.append(repo_path)
print("pwd:")
!pwd
print(f"{repo_path=}")

# Assumes vivarium_research_ciff_sam/ is in sys.path
# import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vp
import model_validation.ciff_sam_results as csr
import model_validation.ciff_sam_plots as csp

%load_ext autoreload
%autoreload 2

!whoami
!date

pwd:
/ihme/homes/ndbs/vivarium_research_ciff_sam/results
repo_path='/ihme/homes/ndbs/vivarium_research_ciff_sam'
ndbs
Mon Oct 25 16:10:23 PDT 2021


# Goal: Calculate reduction in prevalence of MAM, SAM, and MAM+SAM in the two intervention scenarios

# Load data

In [2]:
results = csr.VivariumResults.cleaned_from_model_spec(4.1)
results.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'stunting_state_person_time',
 'population',
 'ylls',
 'ylds',
 'person_time',
 'cause_state_person_time',
 'cause_transition_count']

# Compute wasting prevalence among all ages and all years post-intervention

In [3]:
results.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,sq_lns,wasting_treatment,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,covered,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,0.000000,covered,uncovered,early_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,201.939767,uncovered,covered,early_neonatal
3,female,2022,mild_child_wasting,state_person_time,29,baseline,207.442847,uncovered,uncovered,early_neonatal
...,...,...,...,...,...,...,...,...,...,...
34556,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,0.000000,covered,covered,2_to_4
34557,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,0.000000,covered,uncovered,2_to_4
34558,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,198117.045859,uncovered,covered,2_to_4
34559,male,2026,susceptible_to_child_wasting,state_person_time,946,wasting_treatment,21226.153320,uncovered,uncovered,2_to_4


In [6]:
wasting_prevalence_post_intervention = csr.get_prevalence(
    results,
    state_variable='wasting_state',
    strata=[], # stratifying by nothing computes prevalence among all ages, sexes, and years unless prefiltered
    prefilter_query="year>'2022'", # filter out pre-intervention year
    multiplier=100, # use units of percent
)
wasting_prevalence_post_intervention

,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier,measure
0,29,baseline,mild_child_wasting,21.471650,state_person_time,state_person_time,100,prevalence
1,29,baseline,moderate_acute_malnutrition,8.004558,state_person_time,state_person_time,100,prevalence
2,29,baseline,severe_acute_malnutrition,1.947800,state_person_time,state_person_time,100,prevalence
3,29,baseline,susceptible_to_child_wasting,68.575993,state_person_time,state_person_time,100,prevalence
...,...,...,...,...,...,...,...,...
140,946,wasting_treatment,mild_child_wasting,22.320811,state_person_time,state_person_time,100,prevalence
141,946,wasting_treatment,moderate_acute_malnutrition,6.996664,state_person_time,state_person_time,100,prevalence
142,946,wasting_treatment,severe_acute_malnutrition,1.611424,state_person_time,state_person_time,100,prevalence
143,946,wasting_treatment,susceptible_to_child_wasting,69.071101,state_person_time,state_person_time,100,prevalence


# Compute prevalence of MAM+SAM

In [49]:
# Test to make sure I'm inverting the state-superstate map correctly:
# First list states in each superstate
superstate_to_states = {
    'acute_malnutrition': ['moderate_acute_malnutrition', 'severe_acute_malnutrition'],
    'no_acute_malnutrition': ['susceptible_to_child_wasting', 'mild_child_wasting'],
}
# Now invert the map to get the superstate of each state
state_to_superstate = {
    state: superstate for superstate, states in superstate_to_states.items() for state in states
}
state_to_superstate

{'moderate_acute_malnutrition': 'acute_malnutrition',
 'severe_acute_malnutrition': 'acute_malnutrition',
 'susceptible_to_child_wasting': 'no_acute_malnutrition',
 'mild_child_wasting': 'no_acute_malnutrition'}

In [50]:
def aggregate_wasting_states(df, append=False):
    """"""
    superstate_to_states = {
        'acute_malnutrition': ['moderate_acute_malnutrition', 'severe_acute_malnutrition'],
        'no_acute_malnutrition': ['susceptible_to_child_wasting', 'mild_child_wasting'],
    }
    state_to_superstate = {
        state: superstate for superstate, states in superstate_to_states.items() for state in states
    }
    aggregated_df = (
        df.rename(columns={'wasting_state':'orig_wasting_state'})
        .assign(wasting_state=lambda df: df['orig_wasting_state'].map(state_to_superstate))
        .pipe(vp.marginalize, 'orig_wasting_state')
    )
    if append:
        return df.append(aggregated_df, ignore_index=True)
    else:
        return aggregated_df

aggregate_wasting_states(wasting_prevalence_post_intervention)

,denominator_measure,input_draw,measure,multiplier,numerator_measure,scenario,wasting_state,value
0,state_person_time,29,prevalence,100,state_person_time,baseline,acute_malnutrition,9.952358
1,state_person_time,29,prevalence,100,state_person_time,baseline,no_acute_malnutrition,90.047642
2,state_person_time,29,prevalence,100,state_person_time,sqlns,acute_malnutrition,7.440408
3,state_person_time,29,prevalence,100,state_person_time,sqlns,no_acute_malnutrition,92.559592
...,...,...,...,...,...,...,...,...
68,state_person_time,946,prevalence,100,state_person_time,sqlns,acute_malnutrition,7.743763
69,state_person_time,946,prevalence,100,state_person_time,sqlns,no_acute_malnutrition,92.256237
70,state_person_time,946,prevalence,100,state_person_time,wasting_treatment,acute_malnutrition,8.608088
71,state_person_time,946,prevalence,100,state_person_time,wasting_treatment,no_acute_malnutrition,91.391912


In [51]:
aggregate_wasting_states(wasting_prevalence_post_intervention, append=True)

,input_draw,scenario,wasting_state,value,numerator_measure,denominator_measure,multiplier,measure
0,29,baseline,mild_child_wasting,21.471650,state_person_time,state_person_time,100,prevalence
1,29,baseline,moderate_acute_malnutrition,8.004558,state_person_time,state_person_time,100,prevalence
2,29,baseline,severe_acute_malnutrition,1.947800,state_person_time,state_person_time,100,prevalence
3,29,baseline,susceptible_to_child_wasting,68.575993,state_person_time,state_person_time,100,prevalence
...,...,...,...,...,...,...,...,...
212,946,sqlns,acute_malnutrition,7.743763,state_person_time,state_person_time,100,prevalence
213,946,sqlns,no_acute_malnutrition,92.256237,state_person_time,state_person_time,100,prevalence
214,946,wasting_treatment,acute_malnutrition,8.608088,state_person_time,state_person_time,100,prevalence
215,946,wasting_treatment,no_acute_malnutrition,91.391912,state_person_time,state_person_time,100,prevalence


In [52]:
144+72

216

In [53]:
# Append prevalence of the "superstates" 'acute_malnutrition' and 'no_acute_malnutrition'
wasting_prevalence_post_intervention = aggregate_wasting_states(
    wasting_prevalence_post_intervention, append=True)

In [67]:
csr.to_ordered_categoricals(wasting_prevalence_post_intervention, inplace=True)
vp.describe(wasting_prevalence_post_intervention)['mean'].unstack('wasting_state')

wasting_state                                                                  severe_acute_malnutrition  \
denominator_measure measure    multiplier numerator_measure scenario                                       
state_person_time   prevalence 100        state_person_time baseline                            1.887599   
                                                            wasting_treatment                   1.601461   
                                                            sqlns                               1.460113   

wasting_state                                                                  moderate_acute_malnutrition  \
denominator_measure measure    multiplier numerator_measure scenario                                         
state_person_time   prevalence 100        state_person_time baseline                              7.955648   
                                                            wasting_treatment                     6.907774   
                                                            sqlns                                 6.149561   

wasting_state                                                                  mild_child_wasting  \
denominator_measure measure    multiplier numerator_measure scenario                                
state_person_time   prevalence 100        state_person_time baseline                    21.516838   
                                                            wasting_treatment           22.409269   
                                                            sqlns                       23.021569   

wasting_state                                                                  susceptible_to_child_wasting  \
denominator_measure measure    multiplier numerator_measure scenario                                          
state_person_time   prevalence 100        state_person_time baseline                              68.639915   
                                                            wasting_treatment                     69.081496   
                                                            sqlns                                 69.368758   

wasting_state                                                                  acute_malnutrition  \
denominator_measure measure    multiplier numerator_measure scenario                                
state_person_time   prevalence 100        state_person_time baseline                     9.843247   
                                                            wasting_treatment            8.509235   
                                                            sqlns                        7.609673   

wasting_state                                                                  no_acute_malnutrition  
denominator_measure measure    multiplier numerator_measure scenario                                  
state_person_time   prevalence 100        state_person_time baseline                       90.156753  
                                                            wasting_treatment              91.490765  
                                                            sqlns                          92.390327

# Compute reduction in wasting prevalence

In [54]:
# Compute reduction in prevalence of all wasting states
wasting_prevalence_reduction = vp.averted(
    wasting_prevalence_post_intervention, baseline_scenario='baseline')
wasting_prevalence_reduction

,input_draw,wasting_state,numerator_measure,denominator_measure,multiplier,measure,scenario,subtracted_from,value
0,29,acute_malnutrition,state_person_time,state_person_time,100,prevalence,sqlns,baseline,2.511949
1,29,acute_malnutrition,state_person_time,state_person_time,100,prevalence,wasting_treatment,baseline,1.427008
2,29,mild_child_wasting,state_person_time,state_person_time,100,prevalence,sqlns,baseline,-1.707001
3,29,mild_child_wasting,state_person_time,state_person_time,100,prevalence,wasting_treatment,baseline,-0.956905
...,...,...,...,...,...,...,...,...,...
140,946,severe_acute_malnutrition,state_person_time,state_person_time,100,prevalence,sqlns,baseline,0.640813
141,946,severe_acute_malnutrition,state_person_time,state_person_time,100,prevalence,wasting_treatment,baseline,0.513911
142,946,susceptible_to_child_wasting,state_person_time,state_person_time,100,prevalence,sqlns,baseline,-0.813819
143,946,susceptible_to_child_wasting,state_person_time,state_person_time,100,prevalence,wasting_treatment,baseline,-0.530945


In [66]:
csr.to_ordered_categoricals(wasting_prevalence_reduction, inplace=True)
vp.describe(wasting_prevalence_reduction)['mean'].unstack('wasting_state')

wasting_state                                                                                  severe_acute_malnutrition  \
denominator_measure measure    multiplier numerator_measure scenario          subtracted_from                              
state_person_time   prevalence 100        state_person_time wasting_treatment baseline                          0.286138   
                                                            sqlns             baseline                          0.427486   

wasting_state                                                                                  moderate_acute_malnutrition  \
denominator_measure measure    multiplier numerator_measure scenario          subtracted_from                                
state_person_time   prevalence 100        state_person_time wasting_treatment baseline                            1.047874   
                                                            sqlns             baseline                            1.806088   

wasting_state                                                                                  mild_child_wasting  \
denominator_measure measure    multiplier numerator_measure scenario          subtracted_from                       
state_person_time   prevalence 100        state_person_time wasting_treatment baseline                  -0.892432   
                                                            sqlns             baseline                  -1.504731   

wasting_state                                                                                  susceptible_to_child_wasting  \
denominator_measure measure    multiplier numerator_measure scenario          subtracted_from                                 
state_person_time   prevalence 100        state_person_time wasting_treatment baseline                            -0.441581   
                                                            sqlns             baseline                            -0.728843   

wasting_state                                                                                  acute_malnutrition  \
denominator_measure measure    multiplier numerator_measure scenario          subtracted_from                       
state_person_time   prevalence 100        state_person_time wasting_treatment baseline                   1.334012   
                                                            sqlns             baseline                   2.233574   

wasting_state                                                                                  no_acute_malnutrition  
denominator_measure measure    multiplier numerator_measure scenario          subtracted_from                         
state_person_time   prevalence 100        state_person_time wasting_treatment baseline                     -1.334012  
                                                            sqlns             baseline                     -2.233574

# Compute percent reduction from baseline

## Doesn't work with my existing `ratio` function because numerator and denominator are automatically matched on `'scenario'` column

## To solve the above problem, write a function wrapper to reset global index columns so I can compare scenarios

In [34]:
def use_custom_index(func, index_cols):
    """"""
    def custom_index_wrapped_function(*args, **kwargs):
        orig_index_cols = vp.INDEX_COLUMNS
        vp.set_global_index_columns(vp.list_columns(index_cols))
        try:
            return func(*args, **kwargs)
        finally:
            # Make sure we reset the global index columns to their original values even if an error is raised
            vp.set_global_index_columns(orig_index_cols)
    return custom_index_wrapped_function

In [69]:
print(vp.INDEX_COLUMNS)
wasting_prevalence_percent_reduction = use_custom_index(vp.ratio, 'input_draw')(
    wasting_prevalence_reduction,
    (wasting_prevalence_post_intervention
     .query("scenario=='baseline'")
     .rename(columns={'scenario': 'reference_scenario'})),
    strata=['wasting_state'],
    numerator_broadcast = 'scenario',
    denominator_broadcast = 'reference_scenario',
    multiplier = 100,
)
print(vp.INDEX_COLUMNS) # Make sure INDEX_COLUMNS got reset to correct value
wasting_prevalence_percent_reduction

['input_draw', 'scenario']
['input_draw', 'scenario']


,wasting_state,input_draw,scenario,reference_scenario,value,numerator_measure,denominator_measure,multiplier
0,acute_malnutrition,29,wasting_treatment,baseline,14.338396,prevalence,prevalence,100
1,acute_malnutrition,29,sqlns,baseline,25.239740,prevalence,prevalence,100
2,acute_malnutrition,223,wasting_treatment,baseline,9.873768,prevalence,prevalence,100
3,acute_malnutrition,223,sqlns,baseline,22.056006,prevalence,prevalence,100
...,...,...,...,...,...,...,...,...
140,susceptible_to_child_wasting,829,wasting_treatment,baseline,-0.607729,prevalence,prevalence,100
141,susceptible_to_child_wasting,829,sqlns,baseline,-1.015437,prevalence,prevalence,100
142,susceptible_to_child_wasting,946,wasting_treatment,baseline,-0.774648,prevalence,prevalence,100
143,susceptible_to_child_wasting,946,sqlns,baseline,-1.187361,prevalence,prevalence,100


In [70]:
csr.to_ordered_categoricals(wasting_prevalence_percent_reduction, inplace=True)
vp.describe(wasting_prevalence_percent_reduction)['mean'].unstack('wasting_state')

wasting_state                                                                          severe_acute_malnutrition  \
denominator_measure multiplier numerator_measure reference_scenario scenario                                       
prevalence          100        prevalence        baseline           wasting_treatment                  15.025312   
                                                                    sqlns                              22.514591   

wasting_state                                                                          moderate_acute_malnutrition  \
denominator_measure multiplier numerator_measure reference_scenario scenario                                         
prevalence          100        prevalence        baseline           wasting_treatment                    13.175274   
                                                                    sqlns                                22.698299   

wasting_state                                                                          mild_child_wasting  \
denominator_measure multiplier numerator_measure reference_scenario scenario                                
prevalence          100        prevalence        baseline           wasting_treatment           -4.148225   
                                                                    sqlns                       -6.993174   

wasting_state                                                                          susceptible_to_child_wasting  \
denominator_measure multiplier numerator_measure reference_scenario scenario                                          
prevalence          100        prevalence        baseline           wasting_treatment                     -0.643362   
                                                                    sqlns                                 -1.061981   

wasting_state                                                                          acute_malnutrition  \
denominator_measure multiplier numerator_measure reference_scenario scenario                                
prevalence          100        prevalence        baseline           wasting_treatment           13.548713   
                                                                    sqlns                       22.679113   

wasting_state                                                                          no_acute_malnutrition  
denominator_measure multiplier numerator_measure reference_scenario scenario                                  
prevalence          100        prevalence        baseline           wasting_treatment              -1.479762  
                                                                    sqlns                          -2.477671

In [71]:
vp.describe(wasting_prevalence_percent_reduction)[['mean', '2.5%', '97.5%']].unstack('wasting_state')

mean  \
wasting_state                                                                         severe_acute_malnutrition   
denominator_measure multiplier numerator_measure reference_scenario scenario                                      
prevalence          100        prevalence        baseline           wasting_treatment                 15.025312   
                                                                    sqlns                             22.514591   

                                                                                                                   \
wasting_state                                                                         moderate_acute_malnutrition   
denominator_measure multiplier numerator_measure reference_scenario scenario                                        
prevalence          100        prevalence        baseline           wasting_treatment                   13.175274   
                                                                    sqlns                               22.698299   

                                                                                                          \
wasting_state                                                                         mild_child_wasting   
denominator_measure multiplier numerator_measure reference_scenario scenario                               
prevalence          100        prevalence        baseline           wasting_treatment          -4.148225   
                                                                    sqlns                      -6.993174   

                                                                                                                    \
wasting_state                                                                         susceptible_to_child_wasting   
denominator_measure multiplier numerator_measure reference_scenario scenario                                         
prevalence          100        prevalence        baseline           wasting_treatment                    -0.643362   
                                                                    sqlns                                -1.061981   

                                                                                                          \
wasting_state                                                                         acute_malnutrition   
denominator_measure multiplier numerator_measure reference_scenario scenario                               
prevalence          100        prevalence        baseline           wasting_treatment          13.548713   
                                                                    sqlns                      22.679113   

                                                                                                             \
wasting_state                                                                         no_acute_malnutrition   
denominator_measure multiplier numerator_measure reference_scenario scenario                                  
prevalence          100        prevalence        baseline           wasting_treatment             -1.479762   
                                                                    sqlns                         -2.477671   

                                                                                                           2.5%  \
wasting_state                                                                         severe_acute_malnutrition   
denominator_measure multiplier numerator_measure reference_scenario scenario                                      
prevalence          100        prevalence        baseline           wasting_treatment                  8.214821   
                                                                    sqlns                             18.179771   

                                                                                                                   \
wasting_state          

# Filter to the desired states (SAM, MAM, SAM+MAM) and display results

In [73]:
states_of_interest = ['severe_acute_malnutrition', 'moderate_acute_malnutrition', 'acute_malnutrition']
vp.describe(
    wasting_prevalence_percent_reduction.query("wasting_state in @states_of_interest")
)[['mean', '2.5%', '97.5%']]

mean  \
denominator_measure multiplier numerator_measure reference_scenario scenario          wasting_state                            
prevalence          100        prevalence        baseline           wasting_treatment severe_acute_malnutrition    15.025312   
                                                                                      moderate_acute_malnutrition  13.175274   
                                                                                      acute_malnutrition           13.548713   
                                                                    sqlns             severe_acute_malnutrition    22.514591   
                                                                                      moderate_acute_malnutrition  22.698299   
                                                                                      acute_malnutrition           22.679113   

                                                                                                                        2.5%  \
denominator_measure multiplier numerator_measure reference_scenario scenario          wasting_state                            
prevalence          100        prevalence        baseline           wasting_treatment severe_acute_malnutrition     8.214821   
                                                                                      moderate_acute_malnutrition  10.610585   
                                                                                      acute_malnutrition           10.434936   
                                                                    sqlns             severe_acute_malnutrition    18.179771   
                                                                                      moderate_acute_malnutrition  19.504417   
                                                                                      acute_malnutrition           19.671876   

                                                                                                                       97.5%  
denominator_measure multiplier numerator_measure reference_scenario scenario          wasting_state                           
prevalence          100        prevalence        baseline           wasting_treatment severe_acute_malnutrition    22.682156  
                                                                                      moderate_acute_malnutrition  16.478265  
                                                                                      acute_malnutrition           16.482414  
                                                                    sqlns             severe_acute_malnutrition    29.385074  
                                                                                      moderate_acute_malnutrition  25.397369  
                                                                                      acute_malnutrition           25.198779

# Now repeat all the above steps but for ages 6-59 months in year 2026